In [ ]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os

In [ ]:
# page_info = fs.get_page_info(
#     account=FANPAGE_LINK,
#     timeout=60,
# )
# page_info

In [ ]:
# if "top_post" in page_info:
#     page_info.pop("top_post")
# page_info_df = pd.DataFrame([page_info])
# page_info_df.to_csv(os.path.join(DATA_FOLDER_PATH, FANPAGE_LINK, "raw", "page_info.csv"), index=False)
# page_info_df

In [ ]:
FANPAGE_LINK = "vebay69"
DATA_FOLDER_PATH = "data"
NUMBER_POST = 10
resume_post_url_file_path = os.path.join(DATA_FOLDER_PATH, FANPAGE_LINK, "url", "resume_post_url.txt")

In [ ]:
def handle_pagination_url(url):
    global resume_post_url
    resume_post_url = url
    print(f"Resume url: {url}")
    utils.append_url_to_history(
        url=resume_post_url,
        file_path=os.path.join(DATA_FOLDER_PATH, FANPAGE_LINK, "url", "post_url_history.txt"),
    )

In [ ]:
resume_post_url = utils.read_url_file(file_path=resume_post_url_file_path)
print(f"Resume post url: {resume_post_url}")
post_list = []

In [ ]:
fs.set_cookies('./cookies/cookies_1.txt')

In [ ]:
# Test cookies
# fs.get_profile(account="mm.musashi")

In [ ]:
try:
    for post in fs.get_posts(
        account=FANPAGE_LINK,
        page_limit=NUMBER_POST//10,
        start_url=resume_post_url,
        request_url_callback=handle_pagination_url,
        options={
            "allow_extra_requests": True,
            "reactions": True,
        },
        timeout=120,
    ):
        print(post)
        post_list.append(post)
        utils.sleep(np.random.randint(5, 10))
except fs.exceptions.TemporarilyBanned:
    print("Error: Temporarily Banned")

except fs.exceptions.AccountDisabled:
    print("Error: Account Disabled")

except Exception as e:
    print(e)

In [ ]:
len(post_list), NUMBER_POST

In [ ]:
utils.write_url_file(
    file_path=resume_post_url_file_path,
    url=resume_post_url,
)
if post_list:
    utils.save_data(
        data_list=post_list,
        type="posts",
        folder_path=os.path.join(DATA_FOLDER_PATH, FANPAGE_LINK, "raw"),
    )